In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F

class ExplainerBase(object):

    def __init__(self, model_interface, data_interface):

        self.model_interface = model_interface
        self.data_interface = data_interface

    def generate_counterfactuals(self):

        raise NotImplementedError

    def opposing_class_constraint(self):
        
        raise NotImplementedError
        
        
        
class TimeseriesCounterfactualGenerator(ExplainerBase):
    def __init__(self, data_interface, model_interface):
        super().__init__(data_interface, model_interface)

    def generate_counterfactuals(self, query_instances, time_constant_index=None, time_constant_diff=None, feature_weights=None, optimizer=None, lr=0.01, max_iter=1000):
        query_instance = query_instances[-1]
        query_instance = torch.FloatTensor(query_instance)
        num_features = len(query_instance)
        cf_initialize = query_instance.clone().detach()  # Start with the query instance

        if feature_weights is None:
            feature_weights = torch.ones(num_features)
        else:
            feature_weights = torch.FloatTensor(feature_weights)

        if optimizer == "adam":
            optim = torch.optim.Adam([cf_initialize], lr, betas=(0.9, 0.999))
        else:
            optim = torch.optim.SGD([cf_initialize], lr, momentum=0.9)  

        target_prob_threshold = 0.5  # Set the desired threshold
        target_class_prob = torch.FloatTensor([target_prob_threshold])

        for i in range(max_iter):
            cf_initialize.requires_grad = True
            optim.zero_grad()
            cf_prob = self.model_interface(cf_initialize)
            # Calculate the loss
            loss = self.opposing_class_constraint(cf_prob, target_class_prob, query_instances, cf_initialize)
            loss.backward()
            optim.step()

        return cf_initialize

    def opposing_class_constraint(self, cf_prob, target_class_prob, query_instances, cf_initialize):
    # Calculate a loss to ensure that the generated instance belongs to the opposing class and; 
    # the distance between query instance and the counterfactual is minimal

        cf_prob = F.sigmoid(cf_prob)

        target_class_loss = F.mse_loss(cf_prob, target_class_prob)

        
        distances = np.linalg.norm(query_instances.detach().numpy() - cf_initialize.detach().numpy(), axis=1)

        # Define the distance decay rate - latter points in time have a greater weighting
        alpha = 0.1 + 0.1 * np.arange(len(query_instances.detach().numpy()))

        # Apply the exponential kernel to assign weights
        weights = np.exp(-alpha * distances)
        normalized_weights = weights / np.sum(weights)

        # Compute the weighted result
        weighted_distance_list = np.dot(normalized_weights, query_instances.detach().numpy())
        weighted_distance = np.sum(weighted_distance_list)
        return target_class_loss + weighted_distance
